In [13]:
import sys
import os
import pandas as pd
import folium
from folium.plugins import HeatMap
import datetime as dt
from PyQt5.QtCore import QUrl, Qt
from PyQt5.QtWidgets import QApplication, QVBoxLayout, QHBoxLayout, QWidget, QSlider, QCheckBox, QScrollArea, QPushButton, QLineEdit, QLabel, QMainWindow, QProgressBar
from PyQt5.QtWebEngineWidgets import QWebEngineView
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from statsmodels.tsa.arima.model import ARIMA

Read data and parse data

In [14]:
df = pd.read_csv('combined_historical_temperatures.csv')

# give loactions cordinates
location_coords = {
    'porvoo': [60.3932, 25.6630],
    'rovaniemi': [66.5039, 25.7294],
    'tampere': [61.4978, 23.7600],
    'turku': [60.4515, 22.2666],
    'vaasa': [63.0960, 21.6158],
    'vantaa': [60.2941, 25.0408],
    'helsinki': [60.1699, 24.9384],
    'jyväskylä': [62.2426, 25.7473],
    'espoo': [60.2055, 24.6559],
    'oulu': [65.0121, 25.4651]
}

# lowercase
df['Location'] = df['Location'].str.lower()

# Map the locations to their coordinates
df['coords'] = df['Location'].map(location_coords)

# convert  to datetime
df['time'] = pd.to_datetime(df['time'])

# Handle missing values
df = df.dropna(subset=['coords', 'tavg'])
# Drop duplicates
df = df.drop_duplicates(subset=['Location', 'time'])

min_date = df['time'].min()
max_date = df['time'].max()

Create a heatmap of finland

In [15]:
def create_heat_map(date, selected_cities):
    data = df[df['time'] == date]
    
    # Create a map centered around Finland
    m = folium.Map(location=[64.0, 26.0], zoom_start=6)
    
    # Create heat map data
    heat_data = [[row['coords'][0], row['coords'][1], row['tavg']] for index, row in data.iterrows() if row['Location'] in selected_cities]
    HeatMap(heat_data).add_to(m)
    
    # Save the map to an HTML file
    map_file = f"heat_map_{date.strftime('%Y-%m-%d')}.html"
    m.save(map_file)
    return map_file

Update the temp graphs

In [16]:
def update_graphs(date, selected_cities, canvas_current, canvas_over_time, canvas_predictions):
    filtered_df = df[df['time'] == date]
    
    # Update the current temperatures graph
    canvas_current.figure.clear()
    ax_current = canvas_current.figure.add_subplot(111)
    filtered_current_df = filtered_df[filtered_df['Location'].isin(selected_cities)]
    ax_current.bar(filtered_current_df['Location'], filtered_current_df['tavg'])
    ax_current.set_title(f'Temperatures on {date.strftime("%Y-%m-%d")}')
    ax_current.set_xlabel('Location')
    ax_current.set_ylabel('Temperature (°C)')
    canvas_current.draw()
    
    # Update the temperatures over time graph
    canvas_over_time.figure.clear()
    ax_over_time = canvas_over_time.figure.add_subplot(111)
    df_filtered_over_time = df[(df['time'] <= date) & (df['Location'].isin(selected_cities))]
    for location, group in df_filtered_over_time.groupby('Location'):
        ax_over_time.plot(group['time'], group['tavg'], label=location)
    ax_over_time.set_title('Temperatures Over Time')
    ax_over_time.set_xlabel('Date')
    ax_over_time.set_ylabel('Temperature (°C)')
    ax_over_time.legend()
    canvas_over_time.draw()
    
    # Clear the SARIMA predictions graph when the date is updated
    canvas_predictions.figure.clear()
    canvas_predictions.draw()

Update heatmap and remove old

In [17]:
def update_view(date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label):
    map_file_path = create_heat_map(date, selected_cities)
    
    # Load the new map
    browser.setUrl(QUrl.fromLocalFile(os.path.abspath(map_file_path)))
    
    # Delete the previous map files
    for file in previous_map_files:
        if os.path.exists(file):
            os.remove(file)
    
    # Keep track of the current map file
    previous_map_files.clear()
    previous_map_files.append(map_file_path)
    
    update_graphs(date, selected_cities, canvas_current, canvas_over_time, canvas_predictions)
    
    # Update the date input and label
    date_input.setText(date.strftime('%Y-%m-%d'))
    date_label.setText(f"Selected Date: {date.strftime('%Y-%m-%d')}")

Sarima prediction for future temps from selected date onwards

In [18]:
def run_arima_prediction(date, selected_cities, canvas_predictions, progress_bar):
    canvas_predictions.figure.clear()
    ax_predictions = canvas_predictions.figure.add_subplot(111)
    
    total_cities = len(selected_cities)
    progress_bar.setMaximum(total_cities)
    
    for i, city in enumerate(selected_cities, 1):
        city_data = df[(df['Location'] == city) & (df['time'] <= date)].set_index('time').asfreq('D').fillna(method='ffill')
        
        if len(city_data) < 2:  # Skip if not enough data for ARIMA
            continue
        
        # Fit the ARIMA model
        model = ARIMA(city_data['tavg'], order=(1, 1, 1),seasonal_order=(1,1,1,12))
        model_fit = model.fit()
        
        # Forecast for the next 1 years
        future_dates = pd.date_range(start=date + dt.timedelta(days=1), periods=365, freq='M')
        forecast = model_fit.get_forecast(steps=len(future_dates))
        forecast_df = forecast.summary_frame()
        forecast_df.index = future_dates
        
        # Print the forecasted values in the console
        print(f"ARIMA Forecast for {city.capitalize()}:")
        print(forecast_df[['mean', 'mean_se', 'mean_ci_lower', 'mean_ci_upper']])
        
        ax_predictions.plot(city_data.index, city_data['tavg'], label=f'{city.capitalize()} Historical')
        ax_predictions.plot(forecast_df.index, forecast_df['mean'], label=f'{city.capitalize()} Forecast')
        
        progress_bar.setValue(i)
    
    ax_predictions.set_title('ARIMA Temperature Predictions')
    ax_predictions.set_xlabel('Date')
    ax_predictions.set_ylabel('Temperature (°C)')
    ax_predictions.legend()
    canvas_predictions.draw()




Update on slider

In [19]:
def on_slider_update(value, min_date, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes):
    selected_date = min_date + dt.timedelta(days=value)
    selected_cities = [city for city, checkbox in checkboxes.items() if checkbox.isChecked()]
    update_view(selected_date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label)

Update on chekbox (does not update map??)

In [20]:
def on_checkbox_update(min_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes):
    selected_date = min_date + dt.timedelta(days=slider.value())
    selected_cities = [city for city, checkbox in checkboxes.items() if checkbox.isChecked()]
    update_view(selected_date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label)

Input box for date update

In [21]:
def on_date_input(date_input, min_date, max_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_label, checkboxes):
    input_date = date_input.text()
    try:
        selected_date = dt.datetime.strptime(input_date, '%Y-%m-%d')
        if min_date <= selected_date <= max_date:
            slider.setValue((selected_date - min_date).days)
            selected_cities = [city for city, checkbox in checkboxes.items() if checkbox.isChecked()]
            update_view(selected_date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label)
        else:
            date_label.setText("Selected Date: Invalid date range")
    except ValueError:
        date_label.setText("Selected Date: Invalid date format")

Force update (not working)

In [22]:
def force_update(min_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes):
    selected_date = min_date + dt.timedelta(days=slider.value())
    selected_cities = [city for city, checkbox in checkboxes.items() if checkbox.isChecked()]
    update_view(selected_date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label)

Exit button (still causes kernel crash)

In [23]:
def exit_application():
    main_window.close()

App

In [24]:
if __name__ == '__main__':
    app = QApplication(sys.argv)

    # Main window setup
    main_window = QMainWindow()
    main_window.setWindowTitle("Heat Map Viewer")
    main_window.setGeometry(100, 100, 1400, 800)

    browser = QWebEngineView()

    # Layout
    main_layout = QVBoxLayout()
    content_layout = QHBoxLayout()

    # Add the browser to the left side
    content_layout.addWidget(browser)

    # Checkbox layout
    checkbox_layout = QVBoxLayout()
    checkboxes = {}
    for city in location_coords.keys():
        checkbox = QCheckBox(city.capitalize())
        checkbox.setChecked(True)
        checkbox.stateChanged.connect(
            lambda state, city=city: on_checkbox_update(min_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes)
        )
        checkboxes[city] = checkbox
        checkbox_layout.addWidget(checkbox)

    scroll_area = QScrollArea()
    scroll_area.setWidgetResizable(True)
    checkbox_container = QWidget()
    checkbox_container.setLayout(checkbox_layout)
    scroll_area.setWidget(checkbox_container)
    content_layout.addWidget(scroll_area)

    graph_widget = QWidget()
    graph_layout = QVBoxLayout(graph_widget)

    # Current temperatures graph
    canvas_current = FigureCanvas(plt.Figure())
    graph_layout.addWidget(canvas_current)

    # Temperatures over time graph
    canvas_over_time = FigureCanvas(plt.Figure())
    graph_layout.addWidget(canvas_over_time)

    # SARIMA predictions graph
    canvas_predictions = FigureCanvas(plt.Figure())
    graph_layout.addWidget(canvas_predictions)

    # Add the graph to the right side
    content_layout.addWidget(graph_widget)

    main_layout.addLayout(content_layout)

    # Date input layout
    date_layout = QVBoxLayout()
    date_input = QLineEdit()
    date_input.setPlaceholderText("YYYY-MM-DD")
    date_input.returnPressed.connect(lambda: on_date_input(date_input, min_date, max_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_label, checkboxes))

    date_label = QLabel("Selected Date:")
    date_layout.addWidget(date_label)
    date_layout.addWidget(date_input)

    main_layout.addLayout(date_layout)

    # Slider
    slider = QSlider(Qt.Horizontal)
    main_layout.addWidget(slider)

    # Force update button
    update_button = QPushButton("Force Update")
    update_button.clicked.connect(lambda: force_update(min_date, slider, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes))
    update_button.setFixedSize(120, 40)
    main_layout.addWidget(update_button, alignment=Qt.AlignCenter)

    # ARIMA update button
    arima_button = QPushButton("Run ARIMA Prediction")
    arima_button.clicked.connect(lambda: run_arima_prediction(min_date + dt.timedelta(days=slider.value()), [city for city, checkbox in checkboxes.items() if checkbox.isChecked()], canvas_predictions, progress_bar))
    arima_button.setFixedSize(160, 40)
    main_layout.addWidget(arima_button, alignment=Qt.AlignCenter)

    # Setup slider
    slider.setMinimum(0)
    slider.setMaximum((max_date - min_date).days)
    slider.setValue(0)
    slider.valueChanged.connect(lambda value: on_slider_update(value, min_date, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label, checkboxes))

    # Add exit button
    exit_button = QPushButton("Exit")
    exit_button.clicked.connect(exit_application)
    exit_button.setFixedSize(100, 40)
    exit_button.setStyleSheet("QPushButton { margin: 10px; }")
    main_layout.addWidget(exit_button, alignment=Qt.AlignRight)

    # Add progress bar
    progress_bar = QProgressBar()
    main_layout.addWidget(progress_bar)

    previous_map_files = []
    selected_cities = [city for city, checkbox in checkboxes.items() if checkbox.isChecked()]
    update_view(min_date, selected_cities, browser, previous_map_files, canvas_current, canvas_over_time, canvas_predictions, date_input, date_label)

    container = QWidget()
    container.setLayout(main_layout)
    main_window.setCentralWidget(container)

    main_window.show()
    sys.exit(app.exec_())


C:\Users\Peik\AppData\Local\Temp\ipykernel_12444\3619631121.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  city_data = df[(df['Location'] == city) & (df['time'] <= date)].set_index('time').asfreq('D').fillna(method='ffill')
c:\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Python311\Lib\site-packages\statsmodels\base\model.py:534: RuntimeWarning: invalid value encountered in scalar 

LinAlgError: Schur decomposition solver error.

SystemExit: 0

C:\Users\Peik\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
